# Introduction to CML data processing - From raw signals to rain rates

<img src="https://wires.onlinelibrary.wiley.com/cms/asset/e241f352-21c4-49e7-a8f7-ecf6ff15686f/wat21337-fig-0001-m.jpg" alt="drawing" width="1300"/>

<img src="https://hess.copernicus.org/articles/24/2931/2020/hess-24-2931-2020-f03-web.png" alt="drawing" width="1000"/>


#### ToC:
* Load and explore CML data
* Perform processing for a CML
* Spotlights of CML processing: Wet-dry detection and wet antenna attenuation
* Bonus: The "nearby-link" approach

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import sys
import os
sys.path.append(os.path.abspath("../poligrain/src"))
sys.path.append(os.path.abspath("../pycomlink/"))

import poligrain as plg
import pycomlink as pycml

## Load OpenMRG example dataset

We have already learned about this in the previous notebook.

In [ ]:
(
    ds_radar1,
    ds_cmls1,
    ds_city_ga1uges,
    ds_gauge_s1mhi,
) = plg.example_data.load_openmrg(data_dir="example_data", subset="8d")


In [ ]:
ds_cmls1.isel(cml_id=123)

In [ ]:
(
    ds_radar,
    ds_cmls,
    ds_city_gauges,
    ds_gauge_smhi,
) = plg.example_data.load_openmrg(data_dir="example_data", subset="8d")

ds_cmls = ds_cmls.rename_vars({'R': 'R_example'})

Note, that this is data for 364 CMLs, each with two sublinks, with a temporal reslotion of 10 sec. For a 8-day period, this is approx. 1.2 GB of data if loaded into memory. CML data can be quite large!

In [ ]:
print(f'{ds_cmls.nbytes/1e6:.2f} MB')

## Prepare data
* Project coordinates to UTM zone for Europe 
* We remove very short CMLs (< 1 km) and CMLs which have frequencies below 30 GHz combined with a length below 2 km. We do this because these CMLs are fairly insensitive to rainfall and this might provide less accurate rainfall estimates. That is not say, that one cannot use rainfall estimates of these CMLs. But since CML data processing and the analysis of the resulting rainfall estimates is not the focus in this example, we remove the more insensitive CMLs.

In [ ]:
# Project coordinates of city gauges
(
    ds_city_gauges.coords["x"],
    ds_city_gauges.coords["y"],
) = plg.spatial.project_point_coordinates(
    ds_city_gauges.lon,
    ds_city_gauges.lat,
    "EPSG:25832",
)

# Project coordinates of radar
(
    ds_radar.coords["x_grid"],
    ds_radar.coords["y_grid"],
) = plg.spatial.project_point_coordinates(
    ds_radar.lon,
    ds_radar.lat,
    "EPSG:25832",
)

# Project coordinates for CMLs
(
    ds_cmls.coords["site_0_x"],
    ds_cmls.coords["site_0_y"],
) = plg.spatial.project_point_coordinates(
    ds_cmls.site_0_lon, ds_cmls.site_0_lat, "EPSG:25832"
)
(
    ds_cmls.coords["site_1_x"],
    ds_cmls.coords["site_1_y"],
) = plg.spatial.project_point_coordinates(
    ds_cmls.site_1_lon, ds_cmls.site_1_lat, "EPSG:25832"
)


# Remove CMLs shorter than 1 km with frequency below 30 GHz
ds_cmls = ds_cmls.where(
    ~((ds_cmls.length / 1000 < 2) & (ds_cmls.frequency / 1000 < 30)), drop=True
)
ds_cmls = ds_cmls.where(ds_cmls.length / 1000 > 1, drop=True)



## Get radar rain rate along each CML for comparison

In [ ]:
get_grid_at_lines = plg.spatial.GridAtLines(
    da_gridded_data=ds_radar,
    ds_line_data=ds_cmls,
)
radar_along_cml = get_grid_at_lines(da_gridded_data=ds_radar.R)

## Explore CML data

### The transmitted and received signal levels (TSL and RSL)

- RSL is the most important variable for rainfall estimation
- TSL can be constant or vary with time if ATPC (automatic transmit power control) is enabled
- RSL and TSL are quantized, often at 0.3 dB for RSL and 1.0 dB for TSL

Note that in some datasets there might be fill values, e.g. -99.9 for RSL which need to be set to NaN. This was already done in this dataset.

In [ ]:
ds_cml = ds_cmls.isel(cml_id=20) 

fig, axs = plt.subplots(2, 1, figsize=(10, 5))
ds_cml.tsl.plot.line(x='time', ax=axs[0])
ds_cml.rsl.plot.line(x='time', ax=axs[1])
axs[1].set_title('');

## Exercise 1
1.1 Plot TSL and RSL time series of other CMLs  
1.2 Calculate `TL`, the difference between TSL and RSL and plot TL with cmls from 1.1

### Exercise 1.1

In [ ]:
# Write you code here..




In [ ]:
if input("Enter 'Solution' to display solutions: ")=='Solution':
    %load solutions/2_1_1_solution.py

### Exercise 1.2

In [ ]:
# Write you code here..




In [ ]:
if input("Enter 'Solution' to display solutions: ")=='Solution':
    %load solutions/2_1_2_solution.py

## Perform all processing steps for a single CML

In [ ]:
ds_cml = ds_cmls.sel(cml_id=10124)

# Calculate Tl (TSL - RSL)
ds_cml['tl'] = ds_cml.tsl - ds_cml.rsl

# Full processing
ds_cml['rsd'] = ds_cml.tl.rolling(time=60*6, center=True).std() 
threshold = 0.4
ds_cml['wet'] = ds_cml.rsd > threshold
ds_cml['baseline'] = pycml.processing.baseline.baseline_constant(
    trsl=ds_cml.tl, 
    wet=ds_cml.wet, 
    n_average_last_dry=5*6)  # use average of TL of last 5 minutes

ds_cml['A_obs'] = ds_cml.tl - ds_cml.baseline
ds_cml["A_obs"] = ds_cml.A_obs.where(ds_cml.A_obs >= 0, 0)
ds_cml['waa'] = pycml.processing.wet_antenna.waa_pastorek_2021_from_A_obs(
    A_obs=ds_cml.A_obs,
    f_Hz=ds_cml.frequency * 1e6,
    pol=ds_cml.polarization.data,
    L_km=ds_cml.length / 1000,
    A_max=3,
    zeta=0.7,  
    d=0.15, 
)

# subtract baseline and wet antenna attenuation from total path loss
ds_cml['A'] = ds_cml.tl - ds_cml.baseline - ds_cml.waa
# set negative values to zero
ds_cml['A'] = ds_cml.A.where(ds_cml.A > 0, 0) 

ds_cml['R'] = pycml.processing.k_R_relation.calc_R_from_A(
    A=ds_cml.A, 
    L_km=float(ds_cml.length)/1000, 
    f_GHz=ds_cml.frequency/1000,
    pol=ds_cml.polarization
)

### ... and plot the results

In [ ]:
# Function to mark wet periods in an existing plot
def highlight_wet_periods(da_wet, ax):
    # set first and last time stamp to NaN
    da_wet = da_wet.copy()
    da_wet = da_wet.where(da_wet.time != da_wet.time.isel(time=0), False)
    da_wet = da_wet.where(da_wet.time != da_wet.time.isel(time=-1), False)
    
    # Get start and end of dry event
    wet_start = np.roll(da_wet, -1) & ~da_wet
    wet_end = np.roll(da_wet, 1) & ~da_wet
    
    # Plot shaded area for each wet event
    for wet_start_i, wet_end_i in zip(
        wet_start.values.nonzero()[0],
        wet_end.values.nonzero()[0],
    ):
        ax.axvspan(da_wet.time.values[wet_start_i], da_wet.time.values[wet_end_i], color='b', alpha=1)

# Plotting code for this exercise

fig, axs = plt.subplots(4, 1, figsize=(12,8), sharex=True)

# Plot RSL of TL and threshold
ds_cml.rsd.plot.line(x='time', ax=axs[0])
axs[0].axhline(threshold, color='k', linestyle='--', label='threshold')

# Plot TL and mark wet periods
ds_cml.tl.plot.line(x='time', ax=axs[1]);
highlight_wet_periods(ds_cml.wet, ax=axs[1])

# Plot TL, baseline and WAA
ds_cml.tl.plot.line(x='time', alpha=0.5, label='tl', ax=axs[2])

axs[2].set_prop_cycle(None)
ds_cml.baseline.plot.line(
    x='time', 
    linestyle=':', 
    label='baseline without WAA', 
    ax=axs[2]
)

axs[2].set_prop_cycle(None)
(ds_cml.baseline + ds_cml.waa).plot.line(
    x='time', 
    label='baseline with WAA', 
    ax=axs[2]
)
axs[2].legend()
axs[2].set_ylabel("baseline and waa")

# Plot CML and radar rain rate
ds_cml.R.plot.line(x='time',label='computed rain rate', ax=axs[3])
radar_along_cml.sel(cml_id=ds_cml.cml_id.data).plot.line(x='time', color='k', label='radar along CML', ax=axs[3])
plt.legend()
axs[2].set_ylabel("rain rate (mm/h)")


plt.xlim(pd.to_datetime('2015-07-25'), pd.to_datetime('2015-07-26'))

for i in range(0, 4):
    axs[i].set_title('')
    if i < 3: axs[i].set_xlabel('');

### Interpolate short gaps in TL

Due to short outages of the data acquisition of the CML data there can be short gaps in TSL and RSL, consequently also in TL. We can interpolate these gaps. The question is, however, up to which length the gaps should be interpolated?

We have found that interpolating gaps in TL up to 5 minutes is still okay.

In [ ]:
ds_cmls['tl'] = ds_cmls.tsl - ds_cmls.rsl
ds_cmls['tl'] = ds_cmls.tl.interpolate_na(dim='time', method='linear', max_gap='5min')

## Spotlights in the CML processing

### Spotlight 1: Wet-dry classification

The goal here is to detect rain-indcued attenuation events so that we can separate them from the fluctuations of TL during dry periods.

- Rainfall generates high TL values for short periods
    
- How can we detect such periods? 

As a first, simple approach, we will use the rolling standard deviation (RSD) of TL and a threshold as suggested by [Schleiß et al. 2010](https://ieeexplore.ieee.org/document/5438724). There are different methods implemented in `pycomlink`. At the end of the notebook we will explore the "nearby-link approach" which works by comparing TL from neighboring CMLs.



#### Compute a rolling standard deviation of TL

In [ ]:
# here we first select one CML
ds_cml = ds_cmls.isel(cml_id=7)

# RSD with a 60-minute window length (note that this is 10sec data)
ds_cml['rsd'] = ds_cml.tl.rolling(time=60*6, center=True).std() 

In [ ]:
fig, axs = plt.subplots(2, 1, figsize=(16,4), sharex=True)
ds_cml.tl.plot.line(x='time', ax=axs[0])
ds_cml.rsd.plot.line(x='time', ax=axs[1])
axs[0].set_ylabel('tl')
axs[1].set_ylabel('RSD')
axs[1].set_title('');

#### Apply a threshold to divide TL into wet (rainy) and dry (non-rainy) periods

In [ ]:
threshold = 0.5
ds_cml = ds_cmls.isel(cml_id=7)

ds_cml['rsd'] = ds_cml.tl.rolling(time=60*6, center=True).std() 
ds_cml['wet'] = ds_cml.rsd > threshold

# Plot RSL of TL and threshold
fig, axs = plt.subplots(2, 1, figsize=(16,4), sharex=True)
ds_cml.rsd.plot.line(x='time', ax=axs[0])
axs[0].axhline(threshold, color='k', linestyle='--', label='threshold')

# Plot TL and mark wet periods
ds_cml.tl.plot.line(x='time', ax=axs[1]);
highlight_wet_periods(ds_cml.wet, ax=axs[1])
axs[0].set_title('')
axs[1].set_title('');

#### Apply a threshold that take the general noisneness of a CML into account 
Method from [Graf et al. 2020](https://hess.copernicus.org/articles/24/2931/2020/hess-24-2931-2020.html) 
that sets the RSD treshold based on a "noisiness indicator", which here is the 80th quantile of the RSD 
(note that higher quantiles e.g. 95th are typically those of RSD during rain events).

In [ ]:
ds_cml = ds_cmls.isel(cml_id=7)
factor = 1.4

ds_cml['rsd'] = ds_cml.tl.rolling(time=60*6, center=True).std() 
ds_cml['rsd_threshold'] = factor * ds_cml.rsd.quantile(.80)
ds_cml['wet'] = ds_cml.rsd > ds_cml.rsd_threshold


# Plot RSL of TL and threshold
fig, axs = plt.subplots(2, 1, figsize=(16,4), sharex=True)
ds_cml.rsd.plot.line(x='time', ax=axs[0])
axs[0].axhline(threshold, color='k', linestyle='--', label='threshold')

# Plot TL and mark wet periods
ds_cml.tl.plot.line(x='time', ax=axs[1]);
highlight_wet_periods(ds_cml.wet, ax=axs[1])
axs[0].set_title('')
axs[1].set_title('');

### Spotlight 2: Estimate wet anteanna attenuation (WAA)
Rain drops on the antenna cover can create additional attenuation that we need to subtract before estimating the path averaged rainfall rate.
<img src="https://amt.copernicus.org/articles/11/4645/2018/amt-11-4645-2018-f14-web.png" alt="drawing" width="1000"/>  
[van Leth et al. 2018](https://doi.org/10.5194/amt-11-4645-2018)

In [ ]:
# do processing of one CML here again to make sure 
# that we all have the same data for the next cells
ds_cml = ds_cmls.sel(cml_id=10124)
threshold = 0.7

ds_cml['rsd'] = ds_cml.tl.rolling(time=60*6, center=True).std() 
ds_cml['wet'] = ds_cml.rsd > threshold

ds_cml['baseline'] = pycml.processing.baseline.baseline_constant(
    trsl=ds_cml.tl, 
    wet=ds_cml.wet, 
    n_average_last_dry=5*6)  # use average of TL of last 5 minutes

#### Time dependet WAA estimation

In [ ]:
# Estimate WAA
ds_cml['waa'] = pycml.processing.wet_antenna.waa_schleiss_2013(
    rsl=ds_cml.tl, 
    baseline=ds_cml.baseline, 
    wet=ds_cml.wet, 
    waa_max=2.0, 
    delta_t=1, 
    tau=15*6, # 15 minutes
)

# subtract baseline and wet antenna attenuation from total path loss
ds_cml['A'] = ds_cml.tl - ds_cml.baseline - ds_cml.waa
# set negative values to zero
ds_cml['A'] = ds_cml.A.where(ds_cml.A > 0, 0) 

# also calc A without WAA compensation for illustration purposes
ds_cml['A_no_waa_correct'] = ds_cml.tl - ds_cml.baseline
ds_cml['A_no_waa_correct'] = ds_cml.A_no_waa_correct.where(ds_cml.A_no_waa_correct > 0, 0)

In [ ]:
def waa_plot(ds_cml, t_start_zoom='2015-07-25 11:00:00', t_end_zoom='2015-07-25 16:00:00'):
    ds_cml_zoom = ds_cml.sel(time=slice(t_start_zoom, t_end_zoom))

    fig, axs = plt.subplots(2, 1, figsize=(16,6), sharex=True)
    ds_cml_zoom.tl.plot.line(x='time', alpha=0.5, label='tl', ax=axs[0])

    axs[0].set_prop_cycle(None)
    ds_cml_zoom.baseline.plot.line(
        x='time', 
        linestyle=':', 
        label='baseline without WAA', 
        ax=axs[0]
    )

    axs[0].set_prop_cycle(None)
    (ds_cml_zoom.baseline + ds_cml_zoom.waa).plot.line(
        x='time', 
        label='baseline with WAA', 
        ax=axs[0]
    )

    ds_cml_zoom.A.plot.line(x='time', label='with WAA', ax=axs[1])
    axs[1].set_prop_cycle(None)
    ds_cml_zoom.A_no_waa_correct.plot.line(x='time', linestyle=':', label='without WAA', ax=axs[1])

    axs[0].set_ylabel('tl (dB)')
    axs[1].set_ylabel('PIA (dB)')
    axs[1].set_title('')
    axs[1].legend()
    axs[0].legend();

waa_plot(ds_cml)

#### Intesity dependent WAA estimation
Method from [Pastorek et al. (2021)](https://doi.org/10.1109/TGRS.2021.3110004) (the method they call "KR-alt"). It is available via the function `pycml.processing.wet_antenna.waa_pastorek_2021_from_A_obs()`. 
  
  Note that this function requires attenuation as input. There is an internal rain-rate dependent WAA estimation which is based on the attenuation that is used as input. You have to pepare e.g. `ds_cml['A_obs'] = ds_cml.tl - ds_cml.baseline` and then pass `ds_cml.A_obs`,which is the observed path-attenuation without WAA correction, into the function to calcualte `ds_cml.waa`. Afterwards you can again use `ds_cml['A'] = ds_cml.tl - ds_cml.baseline - ds_cml.waa` to calculate the rain-induced attenuation which we have called `ds_cml.A` in the example above.

More info on the WAA methods can be found in [this `pycomlink` example notebook](https://github.com/pycomlink/pycomlink/blob/master/notebooks/Wet%20antenna%20attenuation.ipynb)

In [ ]:
ds_cml["A_obs"] = ds_cml.tl - ds_cml.baseline
ds_cml["A_obs"] = ds_cml.A_obs.where(ds_cml.A_obs >= 0, 0)
ds_cml["waa"] = pycml.processing.wet_antenna.waa_pastorek_2021_from_A_obs(
    A_obs=ds_cml.A_obs,
    f_Hz=ds_cml.frequency * 1e6,
    pol=ds_cml.polarization.data,
    L_km=ds_cml.length / 1000,
    A_max=6,
    zeta=0.7,  
    d=0.15, 
)

# calculate attenuation caused by rain and remove negative attenuation
ds_cml["A"] = ds_cml.tl - ds_cml.baseline - ds_cml.waa
ds_cml["A"] = ds_cml.A.where(ds_cml.A > 0, 0)


# Plot 
waa_plot(ds_cml)

## Exercise 2 
2.1 Process one CML with the code from above and use `plg.validation.plot_hexbin`, `plg.validation.calculate_rainfall_metrics` and `plg.validation.plot_confusion_matrix_count` to analyse the CML rain rates compared to radar (using it as reference) for 1h rainfall sums.  
2.2 Reprocess the one CML and change paramters of the wet-dry classification, the WAA estimation or any other part and check the changes in performance compared to the radar

### Exercise 2.1

In [ ]:
# Write code here...




In [ ]:
if input("Enter 'Solution' to display solutions: ")=='Solution':
    %load solutions/2_2_1_solution.py

### Exercise 2.2

In [ ]:
# Write code here...




There is no solution as you can freely play with the parameters.. ;)

# Bonus section: The "nearby-link" approach for wet-dry classification

This method was developed to work with 15-minute min-max data which is typically generated by the "network monotoring systems" (NMS) of CML networks.

The method was originally implemented in the `RAINLINK` package for the R programming language and is described in detail in [Overeem et al. (2016)](https://doi.org/10.5194/amt-9-2425-2016). It has been used in numerous publications, most prominently for the first country-wide CML rainfall maps, derived for the Netherlandsd ([Overeem et al., 2013](https://doi.org/10.1073/pnas.1217961110))

There is now also an implementation available in `pycomlink`, allowing for much easiert and more consisten method intercomparison.

Note: Since we work with the instantaneous 10-sec TSL and RSL data here, we have to generate the 15-minute min-max data first.

In [ ]:
import pycomlink.processing.wet_dry.nearby_wetdry as nearby_wetdry
import pycomlink.processing.nearby_rain_retrival as nearby_rain

In [ ]:
rstl = ds_cmls.rsl - ds_cmls.tsl
pmin = rstl.resample(time="15min").min()
pmax = rstl.resample(time="15min").max()

In [ ]:
ds_cmls

In [ ]:
ds_dist = nearby_wetdry.calc_distance_between_cml_endpoints(
    cml_ids=ds_cmls.cml_id.values,
    site_a_latitude=ds_cmls.site_0_lat,
    site_a_longitude=ds_cmls.site_0_lon,
    site_b_latitude=ds_cmls.site_1_lat,
    site_b_longitude=ds_cmls.site_1_lon,
)

In [ ]:
ds_dist.isel(cml_id1=250).a_to_all_a.plot.hist(bins=50);


In [ ]:
r=15 # radius in km
ds_dist["within_r"] = (
        (ds_dist.a_to_all_a < r)
        & (ds_dist.a_to_all_b < r)
        & (ds_dist.b_to_all_a < r)
        & (ds_dist.b_to_all_b < r)
)

In [ ]:
# hack, because `pmin.length` is used in this function
# but is assumed to be provided in km not in m
pmin['length'] = pmin.length / 1e3

wet, F = nearby_wetdry.nearby_wetdry(
    pmin=pmin,
    ds_dist=ds_dist,
    radius=15,
    thresh_median_P=-1.4,
    thresh_median_PL=-0.7,
    min_links=3,
    interval=15,
    timeperiod=24,
)

In [ ]:
t_start, t_end = "2015-07-27", "2015-07-28 "
for cmlid in [5, 211]:
    (ds_cmls.isel(cml_id=cmlid).tsl - ds_cmls.isel(cml_id=cmlid).rsl).isel(sublink_id=0).sel(
        time=slice(t_start, t_end)
    ).plot(figsize=(12, 3), label="TL: total loss")
    pmin.isel(cml_id=cmlid,sublink_id=0).sel(time=slice(t_start, t_end)).plot(
        label="pmin: 15-minute minimum of rsl - tsl"
    )
    pmax.isel(cml_id=cmlid,sublink_id=0).sel(time=slice(t_start, t_end)).plot(
        label="pmax: 15-minute maximum of rsl - tsl"
    )
    (
        (wet.isel(sublink_id=0,cml_id=cmlid).sel(time=slice(t_start, t_end)) * 140)
        - 70
    ).plot(label="classified wet", alpha=0.5)

    plt.legend()

**Baseline estimation (pref)**  
The baseline is set to the rolling median of the average of pmin and pmax during the last n_average_dry time steps. Default is 24 hours (n_average_dry=96 because 15 minute temporal resolution) and a baseline will be calculated when at least one time step is available (min_periods=1) If pmax is not available e.g. because the min-max data is derived from instanteanous sampled CML data and has the same temporal resolution as the instanteanous CML data, substitute pmax with pmin so pmin and pmax are identical.

In [ ]:
pref = nearby_rain.nearby_determine_reference_level(pmin, pmax, wet, n_average_dry=96, min_periods=1)


**Correction if min-max data**
Correcting pmin and pmax so that no rainfall estimation is carried out during dry time steps. All time steps of pmin which are not classified wet and pmin is smaller than pref are set to pref. Similarly, all time steps of pmax where either the corrected pmin (p_c_min) is not smaller than pref or pmax is not smaller than pref are set to pref. This ensures that only wet time steps are used for rainfall estimation an and that pmax is not above pref which would lead to an overestimation of rainfall.

In [ ]:
p_c_min, p_c_max = nearby_rain.nearby_correct_received_signals(pmin, pmax, wet, pref)


**Calculate rain rates from attenuation data**
Also, correcting for wet antenna attenuation and setting the alpha value which defines how close to the minimum attenuation of each intervall the rain rate should be set.

In [ ]:
R = nearby_rain.nearby_rainfall_retrival(
    pref,
    p_c_min,
    p_c_max,
    F,
    length=pmin.length,
    f_GHz=pmin.frequency / 1e3,
    pol=pmin.polarization,
    waa_max=2.3,
    alpha=0.33,
    F_value_threshold =-32.5,
)

### Compare to RSD wet-dry processing

In [ ]:
# processing with RSD as done above, but now for all CMLs in one `xarray.Dataset`

# calculate wet periods
ds_cmls['roll_std_threshold'] = 1.1 * ds_cmls['tl'].rolling(time=60, center=True).std().quantile(0.85, dim='time')
ds_cmls['wet'] = ds_cmls.tl.rolling(time=60, center=True).std() > ds_cmls.roll_std_threshold

# calculate baseline
ds_cmls["baseline"] = pycml.processing.baseline.baseline_constant(
    trsl=ds_cmls.tl,
    wet=ds_cmls.wet,
    n_average_last_dry=5,
)    

# calculate wet antenna effect
ds_cmls["A_obs"] = ds_cmls.tl - ds_cmls.baseline
ds_cmls["A_obs"] = ds_cmls.A_obs.where(ds_cmls.A_obs >= 0, 0)
ds_cmls["waa"] = pycml.processing.wet_antenna.waa_pastorek_2021_from_A_obs(
    A_obs=ds_cmls.A_obs,
    f_Hz=ds_cmls.frequency * 1e6,
    pol=ds_cmls.polarization.data,
    L_km=ds_cmls.length / 1000,
    A_max=6,
    zeta=0.7,  
    d=0.15, 
)

# calculate attenuation caused by rain and remove negative attenuation
ds_cmls["A"] = ds_cmls.tl - ds_cmls.baseline - ds_cmls.waa
ds_cmls["A"] = ds_cmls.A.where(ds_cmls.A > 0, 0)
# derive rain rate via the k-R relation
ds_cmls["R"] = pycml.processing.k_R_relation.calc_R_from_A(
    A=ds_cmls.A,
    L_km=ds_cmls.length.astype(float) / 1000,  
    f_GHz=ds_cmls.frequency / 1000,  
    pol=ds_cmls.polarization,
)
ds_cmls["R"].data[ds_cmls.R < 0.01] = 0

In [ ]:
ds_cml = ds_cmls.isel(cml_id=123)

fig, ax = plt.subplots(figsize=(12,3))
ds_cml.isel(sublink_id=0).R.plot.line(x='time',label='rain rate with RSD wet-dry')
R.isel(sublink_id=0).sel(cml_id=ds_cml.cml_id.data).plot(label='rain rate with nearby wet-dry')
plt.legend()